In [248]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from IPython.display import display
sb.set() # set the default Seaborn style for graphics

# PROBLEM STATEMENT:

# Most important variable in retaining high Customer Life Value (CLV) customers

## Importing Datasets

In [249]:
customerdata = pd.read_csv("olist_customers_dataset.csv")
orderitems = pd.read_csv("olist_order_items_dataset.csv")
orderpayments = pd.read_csv("olist_order_payments_dataset.csv")
orderreviews = pd.read_csv("olist_order_reviews_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
translation = pd.read_csv("product_category_name_translation.csv")
datasetlist = [orderitems,orderpayments,orderreviews,orders,products,translation]

## Basic Data Exploration

In [250]:
for x in datasetlist:
    x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 

## Checking for Discrepancies

In [251]:
print(len(orderitems.order_id.unique()))
print(len(orders.order_id.unique()))
print(len(orderpayments.order_id.unique()))
print(len(orderreviews.order_id.unique()))

98666
99441
99440
99441


### Missing values of order_id found between the datasets. Filter out the missing values and find out the reason why they are dropped between datasets. 

In [252]:
#Finding list of missing order_id
missinglist = []
testset = set(orderitems.order_id)
for x in orders.order_id:
    if x in testset:
        continue
    else:
        missinglist.append(x)
        

#Extracting missing data from orders dataset
missingdf = pd.DataFrame
missingdf = orders[orders['order_id'].isin(missinglist)]
missingdf.info()
missingdf

<class 'pandas.core.frame.DataFrame'>
Int64Index: 775 entries, 266 to 99415
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       775 non-null    object
 1   customer_id                    775 non-null    object
 2   order_status                   775 non-null    object
 3   order_purchase_timestamp       775 non-null    object
 4   order_approved_at              629 non-null    object
 5   order_delivered_carrier_date   1 non-null      object
 6   order_delivered_customer_date  0 non-null      object
 7   order_estimated_delivery_date  775 non-null    object
dtypes: object(8)
memory usage: 54.5+ KB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
266,8e24261a7e58791d10cb1bf9da94df5c,64a254d30eed42cd0e6c36dddb88adf0,unavailable,2017-11-16 15:09:28,2017-11-16 15:26:57,NaN,NaN,2017-12-05 00:00:00
586,c272bcd21c287498b4883c7512019702,9582c5bbecc65eb568e2c1d839b5cba1,unavailable,2018-01-31 11:31:37,2018-01-31 14:23:50,NaN,NaN,2018-02-16 00:00:00
687,37553832a3a89c9b2db59701c357ca67,7607cd563696c27ede287e515812d528,unavailable,2017-08-14 17:38:02,2017-08-17 00:15:18,NaN,NaN,2017-09-05 00:00:00
737,d57e15fb07fd180f06ab3926b39edcd2,470b93b3f1cde85550fc74cd3a476c78,unavailable,2018-01-08 19:39:03,2018-01-09 07:26:08,NaN,NaN,2018-02-06 00:00:00
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaN,NaN,NaN,2018-09-12 00:00:00
...,...,...,...,...,...,...,...,...
99252,aaab15da689073f8f9aa978a390a69d1,df20748206e4b865b2f14a5eabbfcf34,unavailable,2018-01-16 14:27:59,2018-01-17 03:37:34,NaN,NaN,2018-02-06 00:00:00
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,NaN,NaN,2018-10-01 00:00:00
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,NaN,NaN,2018-09-27 00:00:00
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaN,NaN,2017-09-15 00:00:00


### Missing order_ids found to be cancelled / unsuccessful orders. As such, we will drop these orders from all datasets.

In [253]:
neworders = orders[~orders.order_id.isin(missinglist)]
neworderpayments = orderpayments[~orderpayments.order_id.isin(missinglist)]
neworderreviews = orderreviews[~orderreviews.order_id.isin(missinglist)]

print(len(orderitems.order_id.unique()))
print(len(neworders.order_id.unique()))
print(len(neworderpayments.order_id.unique()))
print(len(neworderreviews.order_id.unique()))

98666
98666
98665
98666


### There is still one missing value in order payments. Let's find out

In [254]:
missinglist = []
testset = set(neworderpayments.order_id)
for x in orderitems.order_id:
    if x in testset:
        continue
    else:
        missinglist.append(x)

#Extracting missing data from orders dataset
missingdf = pd.DataFrame
missingdf = neworderreviews[neworderreviews['order_id'].isin(missinglist)]
missingdf

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
37828,6916ca4502d6d3bfd39818759d55d536,bfbd0f9bdef84302105ad712db648a6c,1,NaN,nao recebi o produto e nem resposta da empresa,2016-10-06 00:00:00,2016-10-07 18:32:28


##### "nao recebi o produto e nem resposta da empresa" =  I did not receive the product and no response from the company
##### So we will drop this order_id as well

##### Seems to have some missing values in delivery date under orders dataset as well

In [255]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


#### Presence of NA values in column 5 and 6. As such, we will drop those too.

In [256]:
x = orders[orders['order_delivered_customer_date'].isna()]
a = x.order_id.tolist()
y = orders[orders['order_delivered_carrier_date'].isna()]
b = x.order_id.tolist()
z = orders[orders['order_approved_at'].isna()]
c = x.order_id.tolist()
d = a+b+c+missinglist
miss = set(d)
len(miss)

2966

In [257]:
newerorders = neworders[~neworders.order_id.isin(miss)]
newerorderpayments = neworderpayments[~neworderpayments.order_id.isin(miss)]
newerorderreviews = neworderreviews[~neworderreviews.order_id.isin(miss)]
newerorderitems = orderitems[~orderitems.order_id.isin(miss)]

print(len(newerorderitems.order_id.unique()))
print(len(newerorders.order_id.unique()))
print(len(newerorderpayments.order_id.unique()))
print(len(newerorderreviews.order_id.unique()))

newerorders[newerorders["order_delivered_customer_date"].isna()]

96475
96475
96475
96475


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [295]:
from datetime import date
 
def numOfDays(date1, date2):
    return (date2-date1).days

datexy = newerorders[["order_id","order_purchase_timestamp","order_delivered_customer_date"]]
datexy.reset_index(inplace=True)
datex=newerorders.order_purchase_timestamp
datey=newerorders.order_delivered_customer_date

year1=datexy.loc[1]["order_purchase_timestamp"][:4]
month1=datexy.loc[1]["order_purchase_timestamp"][5:7]
day1=datexy.loc[1]["order_purchase_timestamp"][8:10]
year2=datexy.loc[1]["order_delivered_customer_date"][:4]
month2=datexy.loc[1]["order_delivered_customer_date"][5:7]
day2=datexy.loc[1]["order_delivered_customer_date"][8:10]
print(numOfDays(date1, date2))

8


In [296]:
count=0
datexy



,index,order_id,order_purchase_timestamp,order_delivered_customer_date
0,0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,2017-10-10 21:25:13
1,1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,2018-08-07 15:27:45
2,2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,2018-08-17 18:06:29
3,3,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,2017-12-02 00:28:42
4,4,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,2018-02-16 18:17:02
...,...,...,...,...
96470,99436,9c5dedf39a927c1b2549525ed64a053c,2017-03-09 09:54:05,2017-03-17 15:08:01
96471,99437,63943bddc261676b46f01ca7ac2f7bd8,2018-02-06 12:58:58,2018-02-28 17:37:56
96472,99438,83c1379a015df1e13d02aae0204711ab,2017-08-27 14:46:43,2017-09-21 11:24:17
96473,99439,11c177c8e97725db2631073c19f07b62,2018-01-08 21:28:27,2018-01-25 23:32:54


In [341]:
delivery_days = []
for x in range(96475):
    year1=datexy.loc[x]["order_purchase_timestamp"][:4]
    month1=datexy.loc[x]["order_purchase_timestamp"][5:7]
    day1=datexy.loc[x]["order_purchase_timestamp"][8:10]
    year2=datexy.loc[x]["order_delivered_customer_date"][:4]
    month2=datexy.loc[x]["order_delivered_customer_date"][5:7]
    day2=datexy.loc[x]["order_delivered_customer_date"][8:10]
    date1 =date(int(year1),int(month1),int(day1))
    date2 =date(int(year2),int(month2),int(day2))
    if numOfDays(date1,date2)!=0:
        delivery_days.append(numOfDays(date1, date2))
        print(numOfDays(date1,date2))
    


8
14
9
14
3
17
10
10
18
13
6
12
5
11
14
18
13
6
13
21
4
5
18
4
29
12
9
14
30
8
19
22
19
2
31
20
8
17
10
12
34
8
9
11
11
4
4
7
13
7
5
19
4
8
3
25
24
6
5
5
7
18
2
10
10
4
5
9
3
6
9
17
20
10
9
7
3
2
18
5
9
21
23
11
13
17
10
24
3
21
16
20
8
14
4
36
11
13
2
6
4
8
10
13
13
11
3
56
1
6
7
13
55
1
2
15
8
21
7
12
6
28
7
2
5
7
5
11
9
3
6
8
8
3
7
7
8
6
8
29
14
16
11
4
5
15
18
9
23
12
8
9
4
4
14
16
14
18
4
5
21
7
8
19
11
8
2
5
5
10
12
13
22
6
8
2
20
20
7
11
10
12
10
12
10
9
5
9
9
8
23
11
4
4
7
26
6
7
9
18
7
8
11
10
2
5
14
20
15
10
8
16
13
13
14
11
8
15
10
8
7
16
10
7
12
10
13
6
11
12
24
10
7
13
10
13
14
1
10
9
7
6
6
20
17
81
9
10
6
6
7
12
4
8
4
17
11
11
23
12
7
15
12
4
14
22
20
2
10
20
23
22
10
5
11
14
25
20
14
13
28
10
2
5
15
9
6
27
11
32
11
14
2
11
6
7
9
16
4
23
2
32
17
8
7
8
20
12
6
6
8
2
16
27
10
30
3
2
13
8
25
15
18
7
12
11
17
14
9
6
3
20
8
11
9
34
7
9
10
2
6
14
12
17
6
11
11
28
8
7
24
3
14
10
8
10
7
7
9
14
11
2
9
19
9
11
11
15
8
20
7
5
26
16
8
10
14
9
24
14
15
9
1
11
12
7
5
18
13
10
10
18
9
3

6
32
5
10
14
4
8
12
6
25
9
10
24
10
11
7
54
9
8
12
10
4
14
19
4
19
7
11
8
12
15
4
20
21
9
19
28
14
16
10
6
8
10
8
16
9
7
28
5
15
13
10
10
11
12
5
4
31
11
6
19
4
18
3
18
6
35
51
49
7
10
7
4
13
12
23
7
15
8
2
9
16
16
1
28
9
7
9
9
12
8
8
6
5
6
6
21
33
9
16
4
7
7
2
8
14
16
8
7
3
11
16
7
11
17
6
70
13
3
15
10
26
18
10
15
7
14
3
10
7
2
11
8
46
6
5
5
10
9
11
8
10
8
8
21
43
8
20
10
3
17
4
2
13
9
5
20
23
9
10
11
27
11
11
20
12
11
7
10
18
6
5
3
6
21
4
23
5
10
7
2
3
3
17
18
10
24
8
7
5
2
14
8
8
6
10
7
20
14
24
11
14
14
32
7
6
11
41
8
7
12
21
7
13
18
16
26
6
9
9
15
2
13
2
2
11
16
1
13
17
20
9
30
5
7
9
17
2
14
9
7
29
8
7
15
10
23
7
12
6
17
5
23
7
14
6
21
7
6
12
6
6
10
10
7
11
12
6
8
6
17
3
11
21
2
10
13
4
13
5
8
6
18
11
8
10
11
9
3
18
8
16
11
22
29
7
12
8
17
10
24
7
18
31
32
11
18
19
9
21
3
5
26
3
24
8
8
13
21
8
13
9
22
2
5
6
6
20
13
3
4
11
4
23
12
13
9
19
9
6
14
9
15
16
21
9
20
10
7
14
9
7
32
5
10
6
19
7
8
22
13
5
5
11
8
2
9
9
5
7
9
7
14
4
16
4
10
3
26
14
13
27
6
12
28
9
10
19
24
9
11
14
48
13
12


13
55
36
9
40
10
1
11
11
8
16
5
3
17
14
11
3
18
19
15
9
5
6
5
22
6
6
5
16
7
10
13
17
8
29
15
17
9
5
7
6
5
19
2
11
18
15
4
20
10
9
12
9
13
2
8
6
19
3
10
10
14
2
2
3
14
24
6
17
8
17
2
46
15
10
21
9
5
8
4
4
5
16
22
6
5
15
4
11
24
5
10
5
8
16
2
17
12
7
7
11
11
5
2
8
21
13
15
7
4
12
7
18
12
5
6
3
16
13
11
25
11
10
11
7
22
12
33
5
9
7
10
8
9
5
10
19
24
3
12
11
8
4
6
20
6
2
2
16
5
17
9
2
6
9
5
8
16
4
3
7
6
6
4
18
15
12
15
18
5
5
7
10
17
18
12
29
8
34
16
21
7
13
3
8
10
7
4
5
18
18
11
3
22
17
10
13
5
2
20
11
18
51
2
8
5
14
14
12
24
5
12
4
17
3
12
16
14
21
10
33
5
7
34
5
9
6
6
5
17
7
12
7
15
13
15
4
80
3
23
6
12
96
6
8
16
17
13
17
10
25
19
16
28
5
8
11
3
9
6
7
17
7
5
9
2
11
4
13
8
8
23
11
10
7
10
14
2
13
15
17
5
27
9
7
14
14
5
16
14
11
6
9
22
14
13
8
3
6
32
39
6
7
24
11
9
22
11
12
12
28
7
10
7
18
23
20
7
34
23
14
8
16
20
26
2
5
32
4
11
31
13
15
5
14
7
6
10
14
10
6
15
11
25
3
10
3
13
1
10
15
8
16
17
18
6
8
3
9
7
8
11
47
14
7
23
10
7
2
20
29
5
7
5
12
18
10
13
4
25
11
8
18
8
4
2
20
6
6
19
37
9
20
2

21
15
5
35
7
5
14
13
4
15
17
9
4
33
18
7
14
19
6
7
25
22
9
4
12
12
11
11
22
11
4
8
8
6
8
7
14
3
19
10
6
16
7
24
9
11
16
12
12
6
3
13
3
12
22
3
8
18
12
8
14
6
16
21
8
12
22
6
11
14
11
38
18
4
13
9
39
14
3
13
22
31
11
5
9
9
27
7
22
23
16
9
9
11
6
13
8
17
50
8
17
7
8
24
5
4
17
6
10
8
9
10
8
6
8
5
15
10
12
16
8
18
5
22
7
10
29
8
12
10
9
10
13
3
25
3
7
7
14
12
11
8
16
49
15
3
8
9
10
20
2
4
20
7
9
13
16
9
6
28
2
12
7
4
5
18
8
8
16
8
12
13
5
12
9
16
19
9
9
28
24
9
8
16
8
10
12
21
9
8
8
8
8
13
4
15
41
12
8
7
14
6
15
9
14
7
7
15
5
17
4
14
9
14
9
35
15
8
12
24
23
2
8
11
19
4
7
2
7
19
7
36
7
18
5
5
9
19
20
47
18
16
26
10
43
8
36
12
6
7
9
25
12
5
13
28
50
19
69
9
6
17
9
7
4
9
19
9
13
12
50
25
29
5
3
5
8
11
5
6
6
6
21
16
4
13
16
9
6
15
5
23
22
46
3
6
13
4
9
31
9
8
5
17
4
12
11
16
16
5
7
10
23
3
5
8
8
11
10
4
9
11
7
2
3
11
10
8
12
10
4
5
3
7
11
57
7
19
7
4
6
9
11
2
2
7
10
3
3
15
6
12
8
10
16
20
6
13
4
2
13
7
26
11
11
14
7
22
5
36
11
15
11
8
18
22
12
5
1
14
8
7
19
8
11
6
9
3
8
31
7
17
9
18
20
7
4
13


27
12
4
10
8
6
10
27
11
16
44
11
15
8
10
10
7
4
13
26
7
14
2
14
15
4
19
7
13
2
4
4
7
17
19
17
21
20
2
12
3
34
16
12
9
7
7
2
25
10
6
24
13
6
8
16
19
25
16
9
12
6
37
4
10
9
11
6
6
6
4
6
15
12
8
7
5
19
10
10
11
14
4
9
35
9
13
9
2
8
20
11
8
5
13
2
6
11
6
14
20
13
5
15
8
8
7
11
6
27
12
49
8
18
13
16
13
11
46
14
11
17
14
7
6
14
4
6
17
27
11
5
16
10
8
4
10
12
7
6
12
29
31
8
9
3
15
9
28
6
4
3
15
7
12
28
10
3
3
4
43
13
4
4
3
2
3
2
13
2
2
2
2
9
25
5
5
6
5
9
14
25
6
16
4
8
2
6
8
8
6
8
8
15
7
5
9
5
10
11
7
35
13
17
34
2
18
7
24
3
28
14
11
7
7
16
28
11
7
20
10
18
2
6
8
17
9
5
14
9
54
13
14
3
24
15
20
4
10
7
22
8
10
10
12
9
3
21
18
29
7
15
7
11
3
6
18
7
4
12
10
4
13
12
7
19
4
16
14
15
2
14
19
41
6
9
2
12
26
14
9
12
3
14
7
7
16
5
10
3
7
12
14
23
12
19
6
8
35
8
16
9
17
7
4
14
13
7
13
9
6
8
10
6
6
10
19
23
10
3
6
21
3
11
11
25
25
8
5
6
10
5
2
34
39
10
19
9
14
10
10
12
8
7
22
5
7
5
17
22
11
7
6
6
13
14
19
2
24
7
14
9
18
6
5
14
26
13
7
12
10
8
10
2
6
14
15
27
58
9
5
4
16
34
19
7
4
5
7
6
6
14
9
7
4
7
24
7

...

4
42
8
10
10
5
16
5
15
18
6
38
11
16
20
7
35
9
26
48
8
9
8
21
14
10
21
8
8
17
8
23
15
10
15
3
5
9
20
8
31
16
4
17
16
5
5
9
23
5
3
6
6
8
31
40
8
16
34
6
5
9
12
9
12
28
16
11
28
7
7
5
13
15
9
6
7
9
14
10
9
6
2
7
9
53
18
31
18
15
9
22
2
17
4
2
18
18
29
13
24
7
11
3
8
12
9
22
19
21
11
22
18
4
12
8
14
15
9
6
10
18
12
19
8
5
23
15
7
4
8
6
8
10
13
39
6
9
14
28
24
6
11
23
1
10
12
7
7
6
13
42
49
18
3
19
4
2
6
8
14
25
26
18
8
8
1
13
39
2
2
17
16
7
18
11
14
11
5
18
15
6
10
16
11
10
18
30
14
10
7
5
3
8
19
32
22
7
27
4
32
41
13
5
10
20
3
8
35
8
13
9
12
37
4
16
4
7
19
9
9
2
8
6
8
21
6
11
10
7
26
10
17
3
12
24
15
13
17
16
10
7
5
10
10
16
3
28
3
6
9
5
18
28
8
6
5
27
4
14
8
10
2
14
9
7
8
7
16
6
11
5
18
11
12
2
23
16
13
32
4
10
8
17
14
9
5
13
18
9
13
12
10
21
6
12
26
2
12
6
13
13
15
14
8
11
13
42
3
10
19
4
14
168
14
8
3
10
8
27
7
21
13
8
6
9
15
9
13
11
8
6
17
22
3
16
10
7
10
5
11
24
9
13
6
11
39
9
9
21
30
13
6
19
2
28
12
11
6
15
5
13
29
4
6
10
7
12
5
2
16
7
17
6
14
7
15
9
9
13
8
10
20
21
10
12
28
50
12


2
1
17
24
14
2
4
19
7
3
11
11
5
8
7
11
14
12
9
31
21
9
10
7
19
7
16
10
29
2
20
24
8
18
15
21
6
11
24
1
9
11
9
15
5
9
9
20
5
6
16
15
13
18
27
18
22
20
5
15
8
13
23
18
12
6
10
7
11
28
4
10
12
4
14
9
10
16
23
8
10
20
15
9
10
17
10
10
10
1
30
7
6
8
7
33
2
16
33
19
7
2
30
13
8
2
16
6
32
15
9
6
30
64
12
12
21
10
19
13
23
12
15
3
3
9
23
30
24
20
8
19
4
10
3
6
1
16
4
7
29
15
14
36
19
1
6
14
12
26
11
13
3
43
9
6
21
14
14
4
19
2
5
7
10
10
5
6
7
8
30
11
4
7
12
21
8
17
6
2
3
12
5
23
12
14
16
6
7
5
6
14
7
6
12
8
1
13
43
6
19
30
2
9
23
5
10
5
22
9
8
8
44
7
5
8
4
11
6
5
16
24
18
3
2
10
14
9
18
21
5
4
5
2
24
7
4
6
8
5
10
16
8
9
7
6
8
8
12
9
16
6
15
29
22
3
10
7
16
9
25
8
44
23
10
14
23
5
7
12
8
7
2
9
18
7
23
6
25
14
8
43
4
6
34
14
35
45
7
41
19
22
11
10
13
10
15
4
9
7
4
17
13
41
12
3
15
19
6
19
19
64
16
5
34
10
22
2
4
14
35
4
30
7
11
29
7
23
4
16
5
7
44
14
10
22
33
4
5
13
8
9
11
8
11
11
5
22
14
7
12
14
28
7
13
34
44
6
12
12
12
7
7
25
25
10
20
5
15
15
17
10
12
16
8
24
11
7
4
5
11
13
9
10
34
34
9
11
12


8
11
9
4
11
14
7
2
14
12
22
8
13
21
7
13
7
10
19
11
3
9
8
16
9
6
7
4
7
9
4
2
8
16
15
7
32
3
19
3
7
7
10
17
11
5
11
17
14
20
6
1
4
8
26
16
6
22
12
7
13
12
7
8
12
13
13
24
14
6
4
11
7
6
14
6
7
22
29
12
7
10
5
6
13
3
8
5
14
12
4
24
25
12
18
11
34
5
9
8
9
8
13
7
3
7
7
2
8
15
22
10
12
21
2
19
16
6
4
2
18
6
12
11
5
2
11
24
17
8
11
10
3
16
2
44
14
11
7
4
24
15
6
11
20
17
11
19
15
9
14
5
17
8
8
6
13
6
8
4
28
12
9
23
26
5
13
7
19
19
7
21
13
18
58
7
16
12
19
32
3
18
23
4
5
13
9
8
14
26
17
45
7
6
7
3
6
8
22
5
19
3
6
6
27
3
5
19
8
11
51
11
17
4
16
7
9
9
10
14
2
27
11
17
14
24
13
8
10
1
7
21
3
15
7
4
5
10
7
17
11
8
9
17
2
14
13
9
12
8
21
19
16
29
11
10
4
6
7
21
11
10
6
9
3
13
1
4
5
11
7
22
4
11
11
2
10
13
3
8
19
3
6
8
10
12
14
9
3
2
9
11
43
15
10
3
8
2
8
9
11
11
14
9
11
7
12
3
13
11
5
6
7
7
32
7
26
11
5
19
45
14
8
3
9
16
19
27
48
6
11
10
18
8
6
21
19
13
17
11
2
12
7
6
6
15
34
7
8
40
16
4
11
7
32
12
7
16
16
9
39
1
8
4
7
7
2
10
8
3
19
13
10
7
8
46
9
15
11
15
12
22
12
8
15
6
4
7
5
8
12
5
35
19
7
9
3
4

10
8
5
7
46
10
17
8
4
12
9
5
8
11
8
7
11
6
4
14
8
8
8
14
32
9
12
4
26
2
9
14
10
3
14
20
6
6
21
10
13
4
6
17
10
22
12
6
20
13
8
4
8
5
3
13
21
14
10
11
11
4
5
21
7
8
5
7
1
17
6
5
17
4
10
64
5
4
3
9
26
12
14
29
10
14
10
17
15
19
15
13
3
26
16
7
13
9
3
9
7
6
8
8
2
15
10
5
8
6
9
11
11
1
9
4
13
12
3
5
12
18
14
5
7
7
6
16
7
19
8
19
30
4
4
18
5
17
10
19
11
14
48
31
5
13
4
11
7
3
11
9
17
11
18
10
11
9
18
7
2
2
12
4
14
6
7
9
23
47
6
3
8
17
6
7
12
16
9
14
4
10
12
5
5
25
28
33
17
4
11
12
59
6
11
10
6
13
5
7
21
10
11
13
8
15
30
7
8
12
6
21
4
10
11
22
3
13
5
7
35
6
8
17
10
5
8
13
26
3
4
16
12
5
12
28
10
11
20
2
11
7
26
9
18
23
2
13
32
30
20
13
6
6
9
5
31
6
15
4
7
8
17
17
20
3
15
19
22
25
10
8
15
2
3
11
12
15
9
29
13
6
13
9
9
2
27
12
2
7
6
17
2
5
13
20
2
8
9
10
13
7
79
14
8
8
20
19
10
25
9
11
7
7
16
8
13
20
13
4
11
16
11
3
6
27
4
4
16
8
32
6
18
24
5
25
6
14
7
7
8
11
10
5
23
17
16
9
20
24
15
8
14
13
4
7
15
8
11
8
4
7
20
15
23
5
11
5
20
40
11
21
9
3
6
20
5
11
24
13
16
12
10
8
21
21
3
5
16
9
3
5
11
2
6


10
13
9
10
14
3
9
13
20
9
11
24
25
12
27
2
10
3
15
23
3
10
11
15
20
3
26
3
2
5
40
13
2
23
26
10
18
50
8
5
10
15
31
8
15
12
5
11
11
8
29
3
9
52
6
15
1
8
16
10
19
10
11
4
11
6
9
7
7
3
6
8
17
4
3
14
8
3
2
6
11
3
10
9
5
5
20
13
3
11
8
13
5
8
13
12
13
4
9
4
10
6
16
25
10
5
3
3
10
3
23
16
1
7
19
5
6
15
11
28
7
7
3
6
10
9
7
9
3
19
5
10
8
7
6
19
5
14
7
9
4
7
105
18
4
7
2
17
5
15
19
35
17
6
8
12
2
20
19
6
4
8
8
14
6
8
15
7
13
12
16
20
6
3
17
10
4
9
6
10
10
10
3
5
5
6
14
8
19
13
11
13
4
3
11
13
10
5
14
14
11
3
11
13
4
11
13
33
16
2
5
4
13
7
5
11
13
3
29
12
7
11
6
11
7
4
8
14
1
5
2
5
16
18
4
23
44
15
30
10
10
13
7
21
7
6
2
11
19
9
10
30
16
15
14
14
9
14
7
6
23
9
14
8
13
8
4
14
22
11
14
28
12
3
36
6
17
12
8
11
29
15
11
3
3
6
19
32
7
7
3
7
23
8
12
3
24
6
10
14
6
9
6
5
2
12
7
16
17
10
38
4
13
8
6
4
7
13
5
32
13
6
13
8
12
26
9
14
7
14
19
17
12
17
6
16
13
4
5
22
9
8
19
5
7
4
38
16
5
8
17
11
4
10
9
20
15
5
11
12
17
6
10
7
18
5
14
6
8
21
8
15
6
8
19
5
37
21
25
13
8
9
15
23
7
5
4
4
28
38
12
23
8
9
2
9
2


In [343]:
delivery_days

[8,
 14,
 9,
 14,
 3,
 17,
 10,
 10,
 18,
 13,
 6,
 12,
 5,
 11,
 14,
 18,
 13,
 6,
 13,
 21,
 4,
 5,
 18,
 4,
 29,
 12,
 9,
 14,
 30,
 8,
 19,
 22,
 19,
 2,
 31,
 20,
 8,
 17,
 10,
 12,
 34,
 8,
 9,
 11,
 11,
 4,
 4,
 7,
 13,
 7,
 5,
 19,
 4,
 8,
 3,
 25,
 24,
 6,
 5,
 5,
 7,
 18,
 2,
 10,
 10,
 4,
 5,
 9,
 3,
 6,
 9,
 17,
 20,
 10,
 9,
 7,
 3,
 2,
 18,
 5,
 9,
 21,
 23,
 11,
 13,
 17,
 10,
 24,
 3,
 21,
 16,
 20,
 8,
 14,
 4,
 36,
 11,
 13,
 2,
 6,
 4,
 8,
 10,
 13,
 13,
 11,
 3,
 56,
 1,
 6,
 7,
 13,
 55,
 1,
 2,
 15,
 8,
 21,
 7,
 12,
 6,
 28,
 7,
 2,
 5,
 7,
 5,
 11,
 9,
 3,
 6,
 8,
 8,
 3,
 7,
 7,
 8,
 6,
 8,
 29,
 14,
 16,
 11,
 4,
 5,
 15,
 18,
 9,
 23,
 12,
 8,
 9,
 4,
 4,
 14,
 16,
 14,
 18,
 4,
 5,
 21,
 7,
 8,
 19,
 11,
 8,
 2,
 5,
 5,
 10,
 12,
 13,
 22,
 6,
 8,
 2,
 20,
 20,
 7,
 11,
 10,
 12,
 10,
 12,
 10,
 9,
 5,
 9,
 9,
 8,
 23,
 11,
 4,
 4,
 7,
 26,
 6,
 7,
 9,
 18,
 7,
 8,
 11,
 10,
 2,
 5,
 14,
 20,
 15,
 10,
 8,
 16,
 13,
 13,
 14,
 11,
 8,
 15,
 10,
 8,
 7,
 16,


In [344]:
delivery=pd.DataFrame(delivery_days)

In [346]:
delivery

,0
0,8
1,14
2,9
3,14
4,3
...,...
96469,8
96470,22
96471,25
96472,17


KeyError: 'order_id'

### Now we will concat these dataframes together

In [ ]:
newerorderitems.info()
newerorders.info()
newerorderpayments.info()
newerorderreviews.info()

In [ ]:
new = pd.merge(newerorderitems, newerorderpayments, on='order_id')
new1 = pd.merge(new, newerorderreviews, on='order_id')
new2 = pd.merge(new1, newerorders, on='order_id')
completedata = pd.merge(new2, products, on='product_id')

# translate category names to english
translate = translation.set_index('product_category_name').to_dict()['product_category_name_english']

completedata['product_category_name'].replace(translate, inplace=True)

print(completedata["product_category_name"])

In [ ]:
completedata.info()

## Still have some null values in product category, description, etc. We will drop all these rows.

In [ ]:
finaldata = completedata.dropna(subset = ['product_category_name','product_name_lenght','product_description_lenght','product_photos_qty','product_weight_g','product_length_cm','product_height_cm','product_width_cm',])
finaldata.info()

### Now we are left with null values only present in the review title and message. We will not be dropping these values

# Calculation of Retention Rates:
### If a customer orders more than once, we will assume that the customer is a "retained customer"
### We will then use a few predictors to determine which variable is most important that impacts the categorical Yes/No of Churn.

In [ ]:
len(finaldata.customer_id.unique())
len(customerdata.customer_unique_id.unique())

"At our system each order is assigned to a unique customerid. This means that the same customer will get different ids for different orders. The purpose of having a customerunique_id on the dataset is to allow you to identify customers that made repurchases at the store." 
### We would need to add a new column, customerunique_id to the orders dataset to identify repeat customers

In [ ]:
customer_translate = customerdata.set_index('customer_id').to_dict()['customer_unique_id']
#finaldata['customer_unique_id'] = finaldata['customer_id'].map(customer_translate)

finaldata['customer_unique_id'] = finaldata.customer_id.map(customer_translate)
len(finaldata.customer_unique_id.unique())
finaldata.info()

In [ ]:
explore = finaldata[['customer_id','customer_unique_id']]
explore.sort_values(by=['customer_unique_id'])

### Extracting rows where unique customer id can be mapped to at least 2 different customer ids  

In [ ]:
retain = explore.drop_duplicates(subset =['customer_id'])
retainlist = retain.customer_unique_id.value_counts()
retainedcustomers = retain[retain.customer_unique_id.isin(retainlist.index[retainlist.ge(2)])]
retainedcustomers

### Making of list of unique customer ids that have ordered > once.

In [ ]:
retaineduniqueid = retainedcustomers.customer_unique_id.unique().tolist()
len(retaineduniqueid)

### Adding a 'retained' column to finaldata

In [ ]:
finaldata['retained'] = np.where(finaldata['customer_unique_id'].isin(retaineduniqueid), True, False)
finaldata


### As you can see, those customers with retained = True have customer_unique_id mapped to at least 2 different customer_id

In [ ]:
explore = finaldata[finaldata['retained'] == True]
explore1 = explore[['customer_id','customer_unique_id','retained']]
explore1.sort_values(by=['customer_unique_id'])

## Lets create a classification tree to find which variable is most important in impacting the Churn (retained = True/False) of a customer

In [ ]:
# Extract Response and Predictors
y = pd.DataFrame(finaldata['retained'])
X = pd.DataFrame(finaldata[['payment_value','product_description_lenght','review_score','product_name_lenght','product_photos_qty']])

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Decision Tree using Train Data
dectree = DecisionTreeClassifier(max_depth = 3)  # create the decision tree object
dectree.fit(X_train, y_train)                    # train the decision tree model


f = plt.figure(figsize=(24,24))
plot_tree(dectree, filled=True, rounded=True, 
          feature_names=X_train.columns, 
          class_names=["False","True"])

## Accuracy measures for train dataset

In [ ]:

# Predict the Response corresponding to Predictors
y_train_pred = dectree.predict(X_train)

# Print the Classification Accuracy
print("Train Data")
print("Accuracy  :\t", dectree.score(X_train, y_train))
print()

# Print the Accuracy Measures from the Confusion Matrix
cmTrain = confusion_matrix(y_train, y_train_pred)
tpTrain = cmTrain[1][1] # True Positives : Y (1) predicted Y (1)
fpTrain = cmTrain[0][1] # False Positives : N (0) predicted Y (1)
tnTrain = cmTrain[0][0] # True Negatives : N (0) predicted N (0)
fnTrain = cmTrain[1][0] # False Negatives : Y (1) predicted N (0)

print("TPR Train :\t", (tpTrain/(tpTrain + fnTrain)))
print("TNR Train :\t", (tnTrain/(tnTrain + fpTrain)))
print()

print("FPR Train :\t", (fpTrain/(tnTrain + fpTrain)))
print("FNR Train :\t", (fnTrain/(tpTrain + fnTrain)))

# Plot the two-way Confusion Matrix
sb.heatmap(confusion_matrix(y_train, y_train_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18})

## Accuracy measures for test dataset

In [ ]:
# Predict the Response corresponding to Predictors
y_test_pred = dectree.predict(X_test)

# Print the Classification Accuracy
print("Test Data")
print("Accuracy  :\t", dectree.score(X_test, y_test))
print()

# Print the Accuracy Measures from the Confusion Matrix
cmTest = confusion_matrix(y_test, y_test_pred)
tpTest = cmTest[1][1] # True Positives : Y (1) predicted Y (1)
fpTest = cmTest[0][1] # False Positives : N (0) predicted Y (1)
tnTest = cmTest[0][0] # True Negatives : N (0) predicted N (0)
fnTest = cmTest[1][0] # False Negatives : Y (1) predicted N (0)

print("TPR Test :\t", (tpTest/(tpTest + fnTest)))
print("TNR Test :\t", (tnTest/(tnTest + fpTest)))
print()

print("FPR Test :\t", (fpTest/(fpTest + tnTest)))
print("FNR Test :\t", (fnTest/(fnTest + tpTest)))

# Plot the two-way Confusion Matrix
sb.heatmap(confusion_matrix(y_test, y_test_pred), 
           annot = True, fmt=".0f", annot_kws={"size": 18})